### <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

#### <div class="license">
<span>License CC BY-NC-SA</span>
<span>Carlos Mandele</span>
</div>

# <h1 style="font-size:250%;text-align:center">Deep Learning:</h1>
## <h1 style="font-size:250%;text-align:center">Aproximando uma lei da natureza diretamente dos dados com redes neurais</h1>
<h4 style="font-size:150%;text-align:center">by Carlos Mandele</h4>
<h5 style="font-size:100%;text-align:center">Data Scientist, in progress</h5>

Aproximando uma lei da natureza diretamente dos dados com Deep Learning


A <a href="https://en.wikipedia.org/wiki/Kleiber%27s_law" target="_blank">lei de Kleiber </a>, formulada pelo biólogo Max Kleiber na década de 1930, afirma que o consumo de energia (metabolismo) dos seres vivos, varia segundo a potência de 3/4 de sua massa corporal. Esta lei funciona de bactérias a baleias, no entanto permanece por enquanto sem explicação física ou geométrica satisfatória. O objetivo deste projeto é prever o metabolismo a partir da massa corporal de uma determinada espécie de seres vivos, usando redes neurais (arquitetura de redes neurais MLP) com o framework Keras/TensorFlow.

### Importação das bibliotecas

In [ ]:
# Importando bibliotecas Python
import tensorflow as tf
import numpy as np
import pandas as pd
print("Bibliotecas python importadas")

Bibliotecas python importadas


## Definindo possibilidades de reprodutibilidade

O desenvolvimento de redes neurais envolve certos processos aleatórios.
Durante os ensaios, é estabelecido temporariamente o estado aleatório para reprodutibilidade, em seguida reitera-se o experimento com diferentes amostragens ou estados aleatórios e obtém-se a média dos resultados.



**NOTA**: para um sistema em produção, você simplesmente revive o estado puramente aleatório com a instrução 'GERME_ALEATÓRIO = "None"

In [ ]:
import os

# Definindo um germe aleatório
GERME_ALEATORIO = 1

# Determinar um estado aleatório para Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATORIO)

# Determinar um estado aleatório para Python random
import random
random.seed(GERME_ALEATORIO)

# Determinar um estado aleatório para NumPy
import numpy as np
np.random.seed(GERME_ALEATORIO)

# Definindo um estado aleatório para TensorFlow
import tensorflow as tf
tf.random.set_seed(GERME_ALEATORIO)

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aleatório fixo")


Germe aleatório fixo


# Aquisição de dados....

#### **Fonte**: http://sites.science.oregonstate.edu/~schaferd/Sleuth/data-sets.html

Ramsey, F., & Schafer, D. (2012). The statistical sleuth: a course in methods of data analysis. Cengage Learning.

## Carregamento da base de dados

In [ ]:
# Criando um diretório para os dados
! mkdir KleiberLaw

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Leitura da base de dados

In [ ]:
# Leitura dos dados
dados_kleiber = pd.read_csv("/content/drive/MyDrive/ColabPPortfoio/KleiberLaw.csv")
print("Leitura dos dados concluida")

Leitura dos dados concluida


## Visualizando os dados

In [ ]:
# Exibindo informações básicas sobre o conjunto de dados
dados_kleiber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  95 non-null     int64  
 1   CommonName  95 non-null     object 
 2   Species     95 non-null     object 
 3   Mass        95 non-null     float64
 4   Metabolism  95 non-null     float64
 5   Lifespan    95 non-null     float64
dtypes: float64(3), int64(1), object(2)
memory usage: 4.6+ KB


O conjunto de dados tem 95 registros e 5 atributos: 'CommonName', 'Species', 'Mass', 'Metabolism', 'Lifespan'

In [ ]:
#Exibindo as n primeiras linhas do conjunto de dados
dados_kleiber.head()

,Unnamed: 0,CommonName,Species,Mass,Metabolism,Lifespan
0,0,Echidna,Tachiglossus aculeatus,2.500,302.0,14.0
1,1,Long-beaked echidna,Zaglossus bruijni,10.300,594.0,20.0
2,2,Platypus,Ornithorhynchus anatinus,1.300,229.0,9.0
3,3,Opossum,Lutreolina crassicaudata,0.812,196.0,5.0
4,4,South American opossum,Didelphis marsupialis,1.330,299.0,6.0


In [ ]:
# Exibindo uma amostra selecionada aleatoriamente de 5 cópias do conjunto de dados
dados_kleiber.sample(n=5,random_state=42)

# mass em kg
# métabolism em kJ

,Unnamed: 0,CommonName,Species,Mass,Metabolism,Lifespan
68,68,Chamois,Rupicapra rupicapra,40.00,3140.0,21.0
22,22,Scaly anteater,Manis javanica,4.22,529.0,11.0
72,72,Wolverine,Gulogulo,12.70,2820.0,15.0
73,73,Badger,Meles meles,11.10,1440.0,16.0
0,0,Echidna,Tachiglossus aculeatus,2.50,302.0,14.0


## Pré-processamento de dados

In [ ]:
# Cria-se um novo DF, deletando as colunas desnecessárias
df_new= dados_kleiber.drop(columns='Unnamed: 0')
df_new

,CommonName,Species,Mass,Metabolism,Lifespan
0,Echidna,Tachiglossus aculeatus,2.500,302.0,14.0
1,Long-beaked echidna,Zaglossus bruijni,10.300,594.0,20.0
2,Platypus,Ornithorhynchus anatinus,1.300,229.0,9.0
3,Opossum,Lutreolina crassicaudata,0.812,196.0,5.0
4,South American opossum,Didelphis marsupialis,1.330,299.0,6.0
...,...,...,...,...,...
90,Rhesus monkey,Macaca mulatta,5.000,960.0,25.0
91,Orangutan,Pongo pygma eus,150.000,15500.0,45.0
92,Gorilla,Gorilla gorilla,250.000,21000.0,45.0
93,Gibbon,Hylobater lar,8.000,1510.0,25.0


### Seleção
Extração de atributos que serão usados para previsão ou variáveis independentes.

**Nota**: o objetivo é tentar prever o metabolismo a partir da massa, portanto, apenas um atributo preditivo, a massa.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Extração de atributos que serão usados para previsão ou variáveis independentes
atributo_preditiv = df_new["Mass"].values.reshape(-1, 1)

# Normalisação do atributo preditivo para facilitar o trabalho do algoritmo
normalizador_atributo_preditiv = StandardScaler()
normalizador_atributo_preditiv.fit(atributo_preditiv)
atributo_preditiv = normalizador_atributo_preditiv.transform(atributo_preditiv)
print("Atributo preditivo extraído e normalizado")

Atributo preditivo extraído e normalizado


Extração do atributo que se procura prever ou atributo alvo ou variável dependente.

Nota: o bjetivo é tentar prever o metabolismo e, portanto, apenas um atributo-alvo, o metabolismo.




In [ ]:
# Extração do atributo a ser previsto ou atributo alvo ou variável dependente
atributo_target = df_new['Metabolism'].values.reshape(-1, 1)

# Normalisação do atributo alvo para facilitar o trabalho do algoritmo
normalizador_atributo_target = StandardScaler()
normalizador_atributo_target.fit(atributo_target)
atributo_target = normalizador_atributo_target.transform(atributo_target)
print("Atributo alvo extraído e normalizado")

Atributo alvo extraído e normalizado


## Construção de uma rede neural tipo MLP (Multilayer Perceptron) com três camadas:
- ### camada de entrada, camada oculta, camada de saída

In [ ]:
#Construção de uma rede neural tipo percptron de três camadas:
#camada de entrada, camada oculta, camada de saída
modelo_rede_neural = tf.keras.models.Sequential([tf.keras.layers.Dense(units=1, input_shape=[1]),
                                                 tf.keras.layers.Dense(units=10, activation='relu'),
                                                 tf.keras.layers.Dense(units=1)])
# Visualizando a arquitetura da rede
print("Arquitetura de redes neurais:")
modelo_rede_neural.summary()

Arquitetura de redes neurais:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 2         
                                                                 
 dense_1 (Dense)             (None, 10)                20        
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


## Compilação da rede neural
- ### otimizador: Adam
- ### taxa de aprendizagem: 0,001
- ### função de erro: erro quadrado médio ('mean_squared_error')



In [ ]:
# Compilação da rede neural - Otimizador: Adam, Taxa de aprendizagem: 0,001,
# Função de erro: Erro quadratico médio (mean_squared_error)
modelo_rede_neural.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                           loss='mean_squared_error')

print("Rede de neurónios compilada")


Rede de neurónios compilada


## Treinando a rede neural em dados:
- ### atributo_preditiv e atributo_target
- ### para 500 iterações ou épocas


**NOTA**: uma época representa a exposição de toda a base de dados de treiamento para a rede (modelo).

In [ ]:
# Treinando a rede neural em dados: Variável predetiva(mass) e variável alvo(metabolism),
# para 500 iterações ou épocas
trail = modelo_rede_neural.fit(atributo_preditiv,atributo_target,epochs=500,verbose=1)
# Exibição de erros no final do treinamento
print("Erro no final:",trail.history['loss'][-1])

Epoch 1/500
3/3 [==============================] - 1s 8ms/step - loss: 1.3312
Epoch 2/500
3/3 [==============================] - 0s 6ms/step - loss: 1.3194
Epoch 3/500
3/3 [==============================] - 0s 6ms/step - loss: 1.3131
Epoch 4/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2971
Epoch 5/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2887
Epoch 6/500
3/3 [==============================] - 0s 7ms/step - loss: 1.2751
Epoch 7/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2689
Epoch 8/500
3/3 [==============================] - 0s 6ms/step - loss: 1.2560
Epoch 9/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2460
Epoch 10/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2343
Epoch 11/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2252
Epoch 12/500
3/3 [==============================] - 0s 5ms/step - loss: 1.2183
Epoch 13/500
3/3 [==============================] - 0s 5ms/st

### Testando o modelo após o treinamento
* A validação da função que a rede neural aprendeu é realizada com os dados de teste que não faziam parte do conjunto de dados de treinamento.

In [ ]:
# Ser humano, Homo sapiens
animal = "ser humano"
mass = 6.50E+01

# Normalisação de dados de entrada
mass_normalisee = normalizador_atributo_preditiv.transform(np.array([mass]).reshape(-1, 1))
verdadeiro_valor_metabolism = 7.56E+03

# Aplicação do modelo na inferência ou predição da variável dependente e "desnormalização" do resultado
predit_metabolism = normalizador_atributo_target.inverse_transform(modelo_rede_neural.predict(mass_normalisee))[0][0]

# Exibição da previsão, valor verdadeiro (medido) e desvio (ou erro) em %
print("\nAnimal:", animal,
      ", Massa:", mass,
      ", previsão do métabolismo:", round(predit_metabolism,2),
      ", Verdadeiro valor do métabolismo:", verdadeiro_valor_metabolism,
      ",% de variância:", round((verdadeiro_valor_metabolism-predit_metabolism)/verdadeiro_valor_metabolism*100,2),"%")


1/1 [==============================] - 0s 105ms/step

Animal: ser humano , Massa: 65.0 , previsão do métabolismo: 8333.68 , Verdadeiro valor do métabolismo: 7560.0 ,% de variância: -10.23 %


In [ ]:
# Gato, Felis silvestris, 3.00E+00, 5.46E+02, 11
animal = "gato"
mass = 3.00E+00

# Normalisação de dados de entrada
mass_normalisee = normalizador_atributo_preditiv.transform(np.array([mass]).reshape(-1, 1))
verdadeiro_valor_metabolism = 5.46E+02

# Aplicação do modelo na inferência ou predição da variável dependente e "desnormalização" do resultado
predit_metabolism = normalizador_atributo_target.inverse_transform(modelo_rede_neural.predict(mass_normalisee))[0][0]

# Exibição da previsão, valor verdadeiro (medido) e desvio (ou erro) em %
print("\nAnimal:", animal,
      ", Massa:", mass,
      ", previsão do métabolismo:", round(predit_metabolism,2),
      ", Verdadeiro valor do métabolismo:", verdadeiro_valor_metabolism,
      ",% de variância:", round((verdadeiro_valor_metabolism-predit_metabolism)/verdadeiro_valor_metabolism*100,2),"%")



1/1 [==============================] - 0s 46ms/step

Animal: gato , Massa: 3.0 , previsão do métabolismo: 507.45 , Verdadeiro valor do métabolismo: 546.0 ,% de variância: 7.06 %


In [ ]:
# Cavalo, Equus cabalus, 4.00E+02, 3.20E+04, 40
animal = "cavalo"
mass = 4.00E+02

# Normalisação de dados de entrada
mass_normalisee = normalizador_atributo_preditiv.transform(np.array([mass]).reshape(-1, 1))
verdadeiro_valor_metabolism  = 3.20E+04

# Aplicação do modelo na inferência ou predição da variável dependente e "desnormalização" do resultado
predit_metabolisme = normalizador_atributo_target.inverse_transform(modelo_rede_neural.predict(mass_normalisee))[0][0]

# Exibição da previsão, valor verdadeiro (medido) e desvio (ou erro) em %
print("\nAnimal:", animal,
      ", Massa:", mass,
      ", previsão do métabolismo:", round(predit_metabolisme,2),
      ", Verdadeiro valor do métabolismo:", verdadeiro_valor_metabolism,
      ",% de variância:", round((verdadeiro_valor_metabolism-predit_metabolisme)/verdadeiro_valor_metabolism*100,2),"%")


1/1 [==============================] - 0s 112ms/step

Animal: cavalo , Massa: 400.0 , previsão do métabolismo: 31611.03 , Verdadeiro valor do métabolismo: 32000.0 ,% de variância: 1.22 %


### Conclusão

Observa-se que a rede neural retorna valores muito próximos dos valores verdadeiros dados pela lei de Kleiber. É importante entender que a rede neural não aprende uma fórmula exata, mas que ela se aproxima iterativamente de uma função.

**NOTA**: A rede neural é capaz de aprender a aproximar uma função diretamente dos dados. O processo de aprendizagem é iterativo.
    

In [ ]:
print("Projeto concluído")

Projeto concluído
